In [ ]:
!pip install pyTelegramBotAPI

In [ ]:
import telebot

In [ ]:
bot = telebot.TeleBot('')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image
import numpy as np

In [ ]:
model = load_model('/content/drive/MyDrive/models/insects-data-32-0.9585.hdf5')

In [ ]:
classes = {0: 'Бабочка', 1: 'Стрекоза', 2: 'Кузнечик', 3: 'Божья коровка', 4: 'Комар'}

In [ ]:
@bot.message_handler(commands=['start', 'help'])
def start(message):
    bot.send_message(message.chat.id, '<b>Бот классификатор насекомых приветствует!</b>', parse_mode='html')
    bot.send_message(message.chat.id, 'Отправьте фото, чтобы узнать, какое насекомое на нём изображено.')

In [ ]:
def analysis_of_photo(dir_path):
    file = image.load_img(dir_path, target_size=(48,48))
    X = image.img_to_array(file)
    X = np.expand_dims(X, axis=0)
    img = np.vstack([X])
    prediction = model.predict(img)
    return classes[np.argmax(prediction)]

In [ ]:
@bot.message_handler(content_types=['photo'])
def response_to_photo(message):
    bot.reply_to(message, 'Анализирую изображение...')
    file_info = bot.get_file(message.photo[len(message.photo) - 1].file_id)
    downloaded_file = bot.download_file(file_info.file_path)
    dir_path = '/content/drive/MyDrive/' + message.photo[1].file_id
    with open(dir_path, 'wb') as new_file:
        new_file.write(downloaded_file)
    analysis = analysis_of_photo(dir_path)
    bot.send_message(message.chat.id, analysis)   

In [ ]:
bot.polling(none_stop=True, interval=0)